In [54]:
# Global variables

cabinet_id = [1,5,8,9,10,20]
path = '/home/boshi/Documents/Science/E2EAff/logs/two_franka_chair_pc_partial_cp_map/ppo_pc_pure/ppo_pc_pure_new5_cp_push_seed-1/map01_3000.pt'
output_path = "images/chair_cp/map_3000"
map_dim = [3,4]
gripper_dim = [5,6] # filter out points belong to grippers

sensor_sep = 60     # the angle between two adjacent sensors
sensor_count = 1    # how many views to take pictures
radius = 2.0          # view distance form center
theta = 60         # view angle from upright axis
sensor_width = 512
sensor_height = 341

spp = 256          # render quality

In [55]:
# Create raw xml file

import numpy as np
import torch
import math

def standardize_bbox(pcl):
    mins = np.amin(pcl, axis=0, keepdims=True)
    maxs = np.amax(pcl, axis=0, keepdims=True)
    # maxs[:, 3:] = np.maximum(maxs[:, 3:], np.ones_like(maxs[:, 3:])*0.1)
    center = ( mins + maxs ) / 2.
    scale_xyz = np.amax(maxs-mins, axis=(0,1))
    scale_color = maxs-mins
    pcl[:, :3] = (pcl[:, :3] - center[:, :3]) / (scale_xyz + 1e-8)
    pcl[:, 3:] = (pcl[:, 3:] - center[:, 3:]) / (scale_color[:, 3:] + 1e-8)
    # print("Center: {}, Scale: {}".format(center, scale))
    # pcl[:, 2] += 0.2
    return pcl

xml_head = \
"""
<scene version="0.6.0">
    <integrator type="path">
        <integer name="maxDepth" value="-1"/>
    </integrator>
    <sensor type="perspective">
        <float name="farClip" value="100"/>
        <float name="nearClip" value="0.1"/>
        <transform name="toWorld">
            <lookat origin="3,3,3" target="0,0,0" up="0,0,1"/>
        </transform>
        <float name="fov" value="25"/>
        
        <sampler type="ldsampler">
            <integer name="sampleCount" value="256"/>
        </sampler>
        <film type="ldrfilm">
            <integer name="width" value="1600"/>
            <integer name="height" value="1200"/>
            <rfilter type="gaussian"/>
            <boolean name="banner" value="false"/>
        </film>
    </sensor>
    
    <bsdf type="roughplastic" id="surfaceMaterial">
        <string name="distribution" value="ggx"/>
        <float name="alpha" value="0.05"/>
        <float name="intIOR" value="1.46"/>
        <rgb name="diffuseReflectance" value="1,1,1"/> <!-- default 0.5 -->
    </bsdf>
    
"""

xml_ball_segment = \
"""
    <shape type="sphere">
        <float name="radius" value="0.018"/>
        <transform name="toWorld">
            <translate x="{}" y="{}" z="{}"/>
        </transform>
        <bsdf type="diffuse">
            <rgb name="reflectance" value="{},{},{}"/>
        </bsdf>
    </shape>
"""

xml_tail = \
"""
    <shape type="rectangle">
        <ref name="bsdf" id="surfaceMaterial"/>
        <transform name="toWorld">
            <scale x="10" y="10" z="1"/>
            <translate x="0" y="0" z="-0.5"/>
        </transform>
    </shape>
    
    <shape type="rectangle">
        <transform name="toWorld">
            <scale x="10" y="10" z="1"/>
            <lookat origin="-4,4,20" target="0,0,0" up="0,0,1"/>
        </transform>
        <emitter type="area">
            <rgb name="radiance" value="6,6,6"/>
        </emitter>
    </shape>
</scene>
"""


def colormap(x,y,z):
    vec = np.array([x,y,z])
    vec = np.clip(vec, 0.001,1.0)
    return [vec[0], vec[1], vec[2]]

if path[-2:] == 'pt':
    all_pcl = np.array(torch.load(path, map_location="cpu").cpu())
else:
    all_pcl = np.load(path)

if isinstance(map_dim, list) and len(map_dim) == 1 :
    map_dim = map_dim[0]

for instance in range(all_pcl.shape[0]):

    xml_segments = [xml_head]

    if isinstance(map_dim, int) and map_dim != -1 :
        pcl = all_pcl[instance]
        pcl = standardize_bbox(pcl)
        color_ = pcl[:, map_dim]
        mx = -100.0
        mn = 100.0
        for i in range(pcl.shape[0]):
            color_[i] = max(color_[i], 0.0)
            value = color_[i]
            mx = max(mx, float(value))
            mn = min(mn, float(value))
        for i in range(pcl.shape[0]):
            value = (color_[i]-mn) / (mx-mn+1e-8)
            if value<0.5:
                r = 0.58
                g = 0.80 - 0.80*value
                b = 1.00 - 1.0*value
            else:
                r = 0.58 + 0.84*(value-0.5)
                g = 0.80 - 0.80*value
                b = 1.00 - 1.0*value
            color = colormap(r, g, b)

            is_gripper = False
            for j in gripper_dim :
                if pcl[i,j] > 0.1:
                    is_gripper = True
                    break
            if is_gripper == False :
                xml_segments.append(xml_ball_segment.format(pcl[i,0],pcl[i,1],pcl[i,2], *color))
        xml_segments.append(xml_tail)

        xml_content = str.join('', xml_segments)

        with open('map/try_map{}.xml'.format(instance), 'w') as f:
            f.write(xml_content)
    
    elif isinstance(map_dim, list) and len(map_dim) == 2:
        pcl = all_pcl[instance]
        pcl = standardize_bbox(pcl)
        color_0 = pcl[:, map_dim[0]]
        color_1 = pcl[:, map_dim[1]]
        mx_0 = -100.0
        mn_0 = 100.0
        mx_1 = -100.0
        mn_1 = 100.0
        for i in range(pcl.shape[0]):
            color_0[i] = max(color_0[i], 0.0)
            value_0 = color_0[i]
            mx_0 = max(mx_0, float(value_0))
            mn_0 = min(mn_0, float(value_0))
            color_1[i] = max(color_1[i], 0.0)
            value_1 = color_1[i]
            mx_1 = max(mx_1, float(value_1))
            mn_1 = min(mn_1, float(value_1))
        for i in range(pcl.shape[0]):
            value_0 = (color_0[i]-mn_0) / (mx_0-mn_0+1e-8)
            value_1 = (color_1[i]-mn_1) / (mx_1-mn_1+1e-8)
            r = 0.58 + 0.42*value_0 - 0.42*value_1
            g = 0.80 - 0.80*value_0 - 0.80*value_1
            b = 1.0 - 1.0*value_0 + 1.0*value_1
            color = colormap(r, g, b)

            is_gripper = False
            for j in gripper_dim :
                if pcl[i,j] > 0.1:
                    is_gripper = True
                    break
            if is_gripper == False :
                xml_segments.append(xml_ball_segment.format(pcl[i,0],pcl[i,1],pcl[i,2], *color))
        xml_segments.append(xml_tail)

        xml_content = str.join('', xml_segments)

        with open('map/try_map{}.xml'.format(instance), 'w') as f:
            f.write(xml_content)




In [56]:
# Defines the image sensor
import mitsuba as mi
import os
mi.set_variant('scalar_rgb')
from mitsuba import ScalarTransform4f as T

def load_sensor(r, phi, theta):
    # Apply two rotations to convert from spherical coordinates to world 3D coordinates.
    origin = T.rotate([0, 0, 1], phi).rotate([0, 1, 0], theta) @ mi.ScalarPoint3f([0, 0, r])

    return mi.load_dict({
        'type': 'perspective',
        'fov': 39.3077,
        'to_world': T.look_at(
            origin=origin,
            target=[0, 0, 0],
            up=[0, 0, 1]
        ),
        'sampler': {
            'type': 'independent',
            'sample_count': 16
        },
        'film': {
            'type': 'hdrfilm',
            'width': sensor_width,
            'height': sensor_height,
            'rfilter': {
                'type': 'tent',
            },
            'pixel_format': 'rgb',
        },
    })

In [57]:
# Load created xml and render

import mitsuba as mi
import os
mi.set_variant('scalar_rgb')

# if dir not exist, create it
if not os.path.exists(os.path.join(output_path)):
    os.makedirs(os.path.join(output_path))
with open(os.path.join(output_path, "cfg.txt"), "w") as f :
    print(path, file=f)

for id in cabinet_id :
    with open("map/try_map{}.xml".format(id), "r") as f:
        with open("map/scene{}.xml".format(id), "w") as w :
            del_begin = False
            for line in f.readlines() :
                if '<film type="ldrfilm">' in line :
                    del_begin = True
                if not del_begin :
                    w.write(line)
                if '</film>' in line :
                    del_begin = False

    scene = mi.load_file("map/scene{}.xml".format(id))

    phis = [sensor_sep * i for i in range(sensor_count)]
    sensors = [load_sensor(radius, phi, theta) for phi in phis]
    images = [mi.render(scene, spp=spp, sensor=sensor) for sensor in sensors]

    for i,image in enumerate(images):   # save images to disk
        mi.util.write_bitmap(os.path.join(output_path, "cabinet_{}_view_{}.png").format(id, i), image)